# GPU / Computing Information

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Aug 20 06:50:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Usually training takes long... write the following code in the inspect of google colab to not allow the google colab server instance to disconnect

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [ ]:
# function ConnectButton() {
#     console.log("Connect Pushed");
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
# }

# setInterval(ConnectButton, 60000);

# Installing dependencies

In [ ]:
# !pip install git+https://github.com/huggingface/transformers.git

In [ ]:
# !pip uninstall tensorflow pytorch

# !pip install tensorflow
# !pip install pytorch

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install transformers==4.17.0
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install datasets==1.18.3


# Downloading and importing dataset

In [ ]:
from datasets import load_dataset, load_metric


dataset = load_dataset("addy88/sanskrit-asr-84", split='train')

training_dataset = load_dataset("addy88/sanskrit-asr-84", split='train[0:5%]')
testing_dataset = load_dataset("addy88/sanskrit-asr-84", split='train[5%:8%]')


# ri = (datasets.ReadInstruction('train', to=10, unit='%') +
#     datasets.ReadInstruction('train', from_=-80, unit='%'))
# train_10_80pct_ds = datasets.load_dataset('bookcorpus', split=ri)

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/260M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/247M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/250M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/253M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/addy88--sanskrit-asr-84-c9e988d1463fc8cc/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


In [ ]:
print("training dataset --->\n\n", training_dataset)
print("\n\ntesting dataset --->\n\n", testing_dataset)

training dataset --->

 Dataset({
    features: ['audioFilename', 'snr', 'duration', 'gender', 'text', 'audio', '__index_level_0__'],
    num_rows: 1861
})


testing dataset --->

 Dataset({
    features: ['audioFilename', 'snr', 'duration', 'gender', 'text', 'audio', '__index_level_0__'],
    num_rows: 1116
})


# Training

## Preprocessing on the dataset

In [ ]:
training_dataset = training_dataset.remove_columns(["audioFilename", "snr", "duration", "gender", "__index_level_0__"])
testing_dataset = testing_dataset.remove_columns(["audioFilename", "snr", "duration", "gender", "__index_level_0__"])

print(len(training_dataset), len(testing_dataset))

1861 1116


In [ ]:
print("training dataset --->\n\n", training_dataset)
print("\n\ntesting dataset --->\n\n", testing_dataset)

training dataset --->

 Dataset({
    features: ['text', 'audio'],
    num_rows: 1861
})


testing dataset --->

 Dataset({
    features: ['text', 'audio'],
    num_rows: 1116
})


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(training_dataset.remove_columns(["audio"]), num_examples=10)

,text
0,प्रापकः परानन्देन आदौ मुख्यः वार्ताः
1,सूच्यते प्रयोग केन्द्रम् सन्ति निर्मितानि प्रति केन्द्रम् स्वत लाभात् न सूच्यते प्रयुक्ताः भवेयुः इति अस्ति सुन्दरि तम्
2,स्वस्य परिवारे कल्याणि मन्त्राः केन अधिकारिणः सूचितम्
3,भारतम् परिहासस्य अष्टाविंशति दिनाङ्के सञ्जाता क्लेशे यामस्य सञ्चालयन् स्थगितवान्
4,श्रीमताम् दिन विवाह्य मन्दिरात् कार्य क्रमेण अपि एतदर्थम् उक्तम् सत् राष्ट्रम् रामनाम कुविन्दाः नद्याम् अद्य राष्ट्रम् पुरस्कारः प्रदास्यति
5,परन्तु अस्माभिः सङ्क्रमणम् पुरतः निरुद्धम् न शक्तम्
6,किम् विशिष्टा अवसरे
7,तेन उक्तम् यत् गते दिने दिव्याम् करुणया पञ्चाशत् धिक् त्रयोदशः सहस्र प्रकरणात् समक्षम् आगतानि सः निवेदितवान् यत् जनैः आवश्यकार्थे एव गृहात् बहिः आगच्छेत्
8,विभिन्ना राजानम् अक्षमः निवेशनम्
9,बन्दी भारतमिति लक्ष्यते


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

# making huggingface datasets for vocab
vocab_train = training_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=training_dataset.column_names)
vocab_test = testing_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=testing_dataset.column_names)

# encoding the vocab in dictionary
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'ई': 0, 'औ': 1, 'च': 2, 'ऊ': 3, 'क': 4, 'र': 5, 'थ': 6, 'ढ': 7, 'आ': 8, 'ठ': 9, 'ह': 10, 'ख': 11, 'अ': 12, 'ॄ': 13, '्': 14, 'ो': 15, 'ळ': 16, 'ौ': 17, 'ु': 18, 'भ': 19, 'व': 20, 'ङ': 21, 'उ': 22, 'ि': 23, 'ल': 24, 'म': 25, 'प': 26, 'इ': 27, 'य': 28, 'द': 29, 'ा': 30, 'ओ': 31, 'ब': 32, 'े': 33, 'ट': 34, 'ृ': 35, 'ज': 36, 'फ': 37, 'ऋ': 38, 'छ': 39, 'ऐ': 40, 'ण': 41, 'ग': 42, 'ध': 43, 'ष': 44, 'ै': 45, 'ः': 46, 'ं': 47, 'त': 48, 'घ': 49, ' ': 50, 'स': 51, 'न': 52, 'ए': 53, 'ऽ': 54, 'श': 55, 'ी': 56, 'ड': 57, 'झ': 58, 'ञ': 59, 'ू': 60}


In [ ]:
# temporary code to get sanskrit dictionary

all_words = set(vocab_train["all_text"][0].split()) | set(vocab_test["all_text"][0].split())
sanskrit_dict = {}

for word in all_words:
  sanskrit_dict[word] = len(word)


In [ ]:
# Adding additional vocab token

print("Initial vocab size :", len(vocab_dict))

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

print("Final vocab size :", len(vocab_dict))

Initial vocab size : 61
Final vocab size : 63


In [ ]:
# dumping the vocab in json file

import json

with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file, ensure_ascii=False)

with open('vocab_ascii.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file, ensure_ascii=True)

with open('sanskrit_dictionary.json', 'w') as sanskrit_dictionary:
  json.dump(sanskrit_dict, sanskrit_dictionary, ensure_ascii=False)

with open('sanskrit_dictionary_ascii.json', 'w') as sanskrit_dictionary:
  json.dump(sanskrit_dict, sanskrit_dictionary, ensure_ascii=True)

## Preprocessing on the dataset (temporary)

In [ ]:
training_dataset = training_dataset.remove_columns(["audioFilename", "snr", "duration", "gender", "__index_level_0__"])
testing_dataset = testing_dataset.remove_columns(["audioFilename", "snr", "duration", "gender", "__index_level_0__"])

print(len(training_dataset), len(testing_dataset))

1861 1116


In [ ]:
print("training dataset --->\n\n", training_dataset)
print("\n\ntesting dataset --->\n\n", testing_dataset)

training dataset --->

 Dataset({
    features: ['text', 'audio'],
    num_rows: 1861
})


testing dataset --->

 Dataset({
    features: ['text', 'audio'],
    num_rows: 1116
})


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(training_dataset.remove_columns(["audio"]), num_examples=10)

,text
0,विनश्यति द्विसहस्त्रम् राजदूतौ सम्मिल्य अभि भावयन् असौ प्रोक्तवान् यत् प्रणिधिः विकासेन वर्तमाने युद्ध प्रणामी परिवर्तित
1,प्राजिता साधने शर्मा आदौ मुख्यः वार्ताः
2,मोक्ष वार्ताः
3,हेन्री सर्वेण जनिता सङ्क्रमणम् आलक्ष्य राज्यस्य करैः क्रियमाणानि उपायात् अभिलक्ष्य नवद्वारम् एकः नियन्त्रणम् स्थापितः
4,प्रधानम् त्रि देशे पानाय उपलब्धम् आर्ते स्थितिम् अभिलक्ष्य अपि चर्चा मनुष्या
5,केन्द्रम् शासनम् द्वारा कावेरी प्रबन्धम् प्रतिकर्म
6,कार्यक्षमम् आयोधनम् स्वामिनः विवेकानन्दः ऐतिहासिकस्थानानाम् कृत वर्षतुल्यम् आलक्ष्य
7,अवधि आस्पदम् अस्ति यत् अस्य निर्माणम् रागम् तत्तुल्यम्
8,राष्ट्रम् सुरक्षितम् केन अजित् भाले अपि हे प्रभावितः क्षेत्रम् निरीक्ष्य कृतमासीत्
9,एक लक्षणानि परीक्षाम् विहितानि


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

# making huggingface datasets for vocab
vocab_train = training_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=training_dataset.column_names)
vocab_test = testing_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=testing_dataset.column_names)

# encoding the vocab in dictionary
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'व': 0, 'न': 1, 'म': 2, 'ल': 3, 'औ': 4, 'उ': 5, 'ळ': 6, 'ै': 7, 'ज': 8, 'स': 9, 'ड': 10, 'ऽ': 11, 'च': 12, 'ः': 13, 'इ': 14, 'ख': 15, 'छ': 16, 'थ': 17, 'ो': 18, 'े': 19, 'श': 20, 'ी': 21, 'अ': 22, 'भ': 23, 'ष': 24, '्': 25, 'घ': 26, 'ग': 27, 'ञ': 28, 'आ': 29, 'ं': 30, 'ई': 31, 'ध': 32, 'ए': 33, 'य': 34, 'द': 35, 'ऊ': 36, 'ट': 37, 'ओ': 38, 'ऋ': 39, 'ठ': 40, 'ि': 41, 'ढ': 42, 'ु': 43, 'ह': 44, 'ब': 45, 'ा': 46, 'झ': 47, 'र': 48, 'ङ': 49, 'ऐ': 50, 'ॄ': 51, 'प': 52, 'ण': 53, 'ृ': 54, 'क': 55, ' ': 56, 'ू': 57, 'त': 58, 'ौ': 59, 'फ': 60}


In [ ]:
# temporary code to get sanskrit dictionary

all_words = set(vocab_train["all_text"][0].split()) | set(vocab_test["all_text"][0].split())
sanskrit_dict = {}

for word in all_words:
  sanskrit_dict[word] = len(word)


In [ ]:
# Adding additional vocab token

print("Initial vocab size :", len(vocab_dict))

vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

print("Final vocab size :", len(vocab_dict))

Initial vocab size : 61
Final vocab size : 63


In [ ]:
# dumping the vocab in json file

import json

with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file, ensure_ascii=False)

with open('vocab_ascii.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file, ensure_ascii=True)

with open('sanskrit_dictionary.json', 'w') as sanskrit_dictionary:
  json.dump(sanskrit_dict, sanskrit_dictionary, ensure_ascii=False)

with open('sanskrit_dictionary_ascii.json', 'w') as sanskrit_dictionary:
  json.dump(sanskrit_dict, sanskrit_dictionary, ensure_ascii=True)

## Starting huggingface transformers code

In [ ]:
# SETTING STUFF UP

# setting up Wav2Vec2 tokenizer

from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

# setting up feature extractor

from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

# setting up Wav2Vec2 processor

from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
processor.save_pretrained("/content/drive/MyDrive/ASR_models/wav2vec2-large-xlsr-hindi-demo/")

In [ ]:
# import torchaudio
import numpy as np

def speech_file_to_array_fn(batch):
    # speech_array, sampling_rate = torchaudio.load(batch["audio_path"])
    batch["speech"] = np.asarray(batch["audio"]["array"]) # speech_array[0].numpy()
    batch["sampling_rate"] = batch["audio"]["sampling_rate"]
    batch["target_text"] = batch["text"]
    return batch

training_dataset = training_dataset.map(speech_file_to_array_fn, remove_columns=training_dataset.column_names)
testing_dataset = testing_dataset.map(speech_file_to_array_fn, remove_columns=testing_dataset.column_names)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
print("training dataset --->\n\n", training_dataset)
print("\n\ntesting dataset --->\n\n", testing_dataset)

training dataset --->

 Dataset({
    features: ['speech', 'sampling_rate', 'target_text'],
    num_rows: 1861
})


testing dataset --->

 Dataset({
    features: ['speech', 'sampling_rate', 'target_text'],
    num_rows: 1116
})


In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(training_dataset))

print(training_dataset[rand_int]["target_text"], "\n")
ipd.Audio(data=np.asarray(training_dataset[rand_int]["speech"]), rate=16000) #autoplay=True)

अपि चोक्तम् यत् परीक्षा पश्चात् अयोजयत् यन्ति अवधेयम् यत् परीक्षा परिणामः गत परीक्षा भारे घोषयित्वा तत्परिहार कल्याणि मन्त्रिणः डाक्टर् हर्षे तदिति 



In [ ]:
rand_int = random.randint(0, len(training_dataset))

print("Target text:", training_dataset[rand_int]["target_text"])
print("Input array shape:", np.asarray(training_dataset[rand_int]["speech"]).shape)
print("Sampling rate:", training_dataset[rand_int]["sampling_rate"])

Target text: पताकायाम् नवे करम् विद्यते
Input array shape: (52320,)
Sampling rate: 16000


## More processing before training

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch



trainings_dataset = training_dataset.map(prepare_dataset, remove_columns=training_dataset.column_names, batch_size=8, num_proc=2, batched=True)
testings_dataset = testing_dataset.map(prepare_dataset, remove_columns=testing_dataset.column_names, batch_size=8, num_proc=2, batched=True)

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

In [ ]:
print(trainings_dataset)
print(testings_dataset)

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1861
})
Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1116
})


In [ ]:
trainings_dataset = trainings_dataset.filter(lambda x: len(x) > 0, input_columns=["labels"])
testings_dataset = testings_dataset.filter(lambda x: len(x) > 0, input_columns=["labels"])

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
print(trainings_dataset)
print(testings_dataset)

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1844
})
Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1097
})


## Preparing for training and then doing training

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.weight', 'project_hid.bias', 'quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.codevectors', 'project_hid.weight', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1712: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/ASR_models/wav2vec2-large-xlsr-hindi-demo",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=1e-4,
  warmup_steps=500,
  save_total_limit=1,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=trainings_dataset,
    eval_dataset=testings_dataset,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


In [ ]:
# Training

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1844
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1740


Step,Training Loss,Validation Loss,Wer
400,8.188100,3.331815,1.000000
800,3.122200,1.977125,1.000000


***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/ASR_models/wav2vec2-large-xlsr-hindi-demo/checkpoint-400
Configuration saved in /content/drive/MyDrive/ASR_models/wav2vec2-large-xlsr-hindi-demo/checkpoint-400/config.json
Model weights saved in /content/drive/MyDrive/ASR_models/wav2vec2-large-xlsr-hindi-demo/checkpoint-400/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/ASR_models/wav2vec2-large-xlsr-hindi-demo/checkpoint-400/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/ASR_models/wav2vec2-large-xlsr-hindi-demo/checkpoint-800
Configuration saved in /content/drive/MyDrive/ASR_models/wav2vec2-large-xlsr-hindi-demo/checkpoint-800/config.json
Model weights saved in /content/drive/MyDrive/ASR_models/wav2vec2-large-xlsr-hindi-demo/checkpoint-800/pytorch_model.bin
Feature extractor saved in /content/drive/

# Testing

## Getting testing dataset

In [ ]:
from datasets import load_dataset, load_metric
testing_dataset = load_dataset("addy88/sanskrit-asr-84-eval", split='train[0:5%]')

In [ ]:
print("\n\ntesting dataset --->\n\n", testing_dataset)



testing dataset --->

 Dataset({
    features: ['audioFilename', 'snr', 'duration', 'gender', 'text', 'audio', '__index_level_0__'],
    num_rows: 465
})


## Preprocessing on testing dataset

In [ ]:
testing_dataset = testing_dataset.remove_columns(["audioFilename", "snr", "duration", "gender", "__index_level_0__"])
print(testing_dataset)

Dataset({
    features: ['text', 'audio'],
    num_rows: 465
})


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(testing_dataset.remove_columns(["audio"]), num_examples=10)

,text
0,विरते अहो रात्रेः पञ्च सप्तम्यधिकरणे तत्र पञ्च त्रिषु सहस्रम् रोगिणः स्वस्ति भूताः
1,अन्यन् मूर्तयः पूर्वान् अनुपादित्सा स्वदेशम् चिकित्सा करणानाम् व्यवस्था
2,अस्मिन् नैव दिवसे भारतस्य साहसिके त्रिभिः अन्तिमात् श्वासः गृहीतः आसीत्
3,देशे अभ्यन्तरे विमाने वाः अस्य मतस्य पञ्च विंशति दिनाङ्के पुनः आरभ्यन्ते
4,बलेन वक्तव्ये उक्तम् यत् मेघा येति तत्
5,तेन अग्रे उक्तम् यत् श्रीरिव मुदे कृषकः सह संवादम् संस्थानम् वाञ्छति अतः तेन कृषकः वार्तालापः आमन्त्रितवान्
6,आक्रमणे यस्मिन् दिवम् तेभ्यः सप्तदश वीर भटेभ्यः तेन श्रद्धाम् जनिः प्रकटिता
7,दुर्घट तदा सञ्जाता यदा निर्वचनम् कार्यार्थान् यानेन जनाः गच्छन्ति स्म अवगाहमानः तावान् क्रमते नयनात् न्यूनम् त्रयोदशः रक्षितः दिवङ्गता
8,उपरिष्टात् श्रीमन् ययोः न्यायात् यत् श्री मुखरैः महता श्रमेण अनुशासने समर्पणे च देशमिव
9,पञ्च सैन्ये कर्मिणः अपि आहतः


In [ ]:
# import torchaudio
import numpy as np

def speech_file_to_array_fn(batch):
    # speech_array, sampling_rate = torchaudio.load(batch["audio_path"])
    batch["speech"] = np.asarray(batch["audio"]["array"]) # speech_array[0].numpy()
    batch["sampling_rate"] = batch["audio"]["sampling_rate"]
    batch["target_text"] = batch["text"]
    return batch

testing_dataset = testing_dataset.map(speech_file_to_array_fn, remove_columns=testing_dataset.column_names)

0ex [00:00, ?ex/s]

In [ ]:
print(testing_dataset)

Dataset({
    features: ['speech', 'sampling_rate', 'target_text'],
    num_rows: 465
})


In [ ]:
testing_dataset = testing_dataset.filter(lambda x: len(x) > 0, input_columns=["target_text"])
print(testing_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['speech', 'sampling_rate', 'target_text'],
    num_rows: 453
})


## Getting Model

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

all_models = [
    "Harveenchadha/vakyansh-wav2vec2-sanskrit-sam-60",         # WER = 77.29
    "addy88/wav2vec2-base-sanskrit-84h",                       # WER = UNABLE TO LOAD
    "addy88/wav2vec2-sanskrit-stt",                            # WER = 77.29
    "KabiAI/wav2vec2-large-xls-r-300m-Sanskrit-Hindi",         # WER = UNABLE TO LOAD
    "Harveenchadha/wav2vec2-pretrained-clsril-23-10k",         # WER = UNABLE TO LOAD
    "frmccann/CLSRIL-23",                                      # WER = 100
    "theainerd/Wav2Vec2-large-xlsr-hindi",                     # WER = 
    "Harveenchadha/vakyansh-wav2vec2-indian-english-enm-700",  # WER = INVALID (Model Works Fine/Well for Porting Sanskrit audio to English)
    "Harveenchadha/vakyansh-wav2vec2-hindi-him-4200"           # WER = 
]

model_str = all_models[8]


processor = Wav2Vec2Processor.from_pretrained(model_str)
model = Wav2Vec2ForCTC.from_pretrained(model_str)

Downloading:   0%|          | 0.00/261 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/729 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

## Finding Wer

In [ ]:
wer = load_metric("wer")

In [ ]:
# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(element):

    global model, processor

    # pad input values and return pt tensor
    input_values = processor(element["speech"], sampling_rate=16_000, return_tensors="pt").input_values
    
    # INFERENCE
    # retrieve logits & take argmax
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)


    # transcribe
    # element["predicted"] = processor.decode(predicted_ids[0])
    element["predicted"] = processor.decode(predicted_ids[0], skip_special_tokens=True)

    return element

In [ ]:
import torch

result = testing_dataset.map(evaluate, batched=False, remove_columns=['speech', 'sampling_rate'])

0ex [00:00, ?ex/s]

In [ ]:
print("WER: {:2f}".format(100 * wer.compute(predictions=result["predicted"], references=result["target_text"])))

WER: 119.330025


In [ ]:
show_random_elements(result, num_examples=20)

,target_text,predicted
0,यथावत् रक्त चत्वारि एकम् अन्तराले पराजिताः,chuthamataraksata chatwari ekam antaralayana parajita
1,त्वाम् विवेश मन्त्रिणाम् उपवेशितम् द्वितीयम् भविष्यति अतः पूर्वम् विगता वर्षे त्वाम् विवेश मन्त्रिणाम् उपदेशनम् संयुक्ते राष्ट्रम् भातः पृथक् आयोजितम् आसीत्,quad videsh amantrenam upavesh anamidam vitiyam bhavishati ata hapurmam vigate varse quad videsh amantrnam uphaveshanam sanyukta rastramaha sabhataha pratag ayojitam asi
2,अक्षिणी च दिने ते आक्रान्ताः,arakshi beca gatadi ne te a kan taha
3,सम्प्रति वार्ताः श्रूयताम् प्राजिता लता आदौ मुखस्य वार्ताः,samprati vartaha shroyantam paravacika lata ado mukhavarta ha
4,प्रधानम् न कापि पूर्व योजना सुरक्षायै हिते च गुरुत्वात् इति इषु सनगरम् गमनेन न खलु जनाः कीदृशम् अपि सुविधाम् अनुभूतवन्तः,radhanamantano nakapi purva yoja naya suraksha chakra dredecha gurdwara iti isho pasara gruham gamanena nakhalu janaha ki drashamaphi a subidham anubhutavantaha
5,वृत्रस्य सततम् पतनम् भवति,mrityu jaras hiar satatam patanam bhawati
6,इति वार्ताः,eity vartaha
7,भारती सेनया निश्चितम् यत् उपयुक्तानि नाम च प्रियायाः कुत्सिताम् समर्थम् प्रकरणम् केन्द्रम् अन्वेषणे विभागाः प्रदास्यति,bharatya senaya nistitam yat upayukta abhyartina chana prakriayaha kutsita atana sambartam prakrnam kendria anuveshanabibagaya pradasyati
8,सूचना प्रसारः मन्त्रिणः प्रकाश जावळी करेण उक्तम् यत् मिथ्या वार्ताः सशङ्कम् ताभ्यः अधिकः यज्ञा अनिष्टा च अस्ति,sucana prasaran mantrina prakasha jawadi karena uptam jatmitya bharta ha sashwalka barta pyah adhikaha bhayajanya anusta cha asti
9,प्रधानम् त्रि श्री नरेन्द्र निर्विद्य चित्रा प्रसङ्गे सङ्गा योग सङ्गतस्य वार्षिक सम्मिल्य भागम् भवितुम् अद्य चिन्ता नगरम् प्रतिष्ठते,pradhanmantry sre narendra modi nija dr divasia sina yatra prasangi sangai sahayuga sanghatanasya barsik sa malani bhagam bhajitam adya singhadao nagaram pratesta de


# Inference

In [ ]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import argparse

def parse_transcription(model, wav_file=None, audio_input=None, sample_rate=None):
    # load pretrained model
    processor = Wav2Vec2Processor.from_pretrained(model)
    model = Wav2Vec2ForCTC.from_pretrained(model)

    # load audio
    if wav_file:
      audio_input, sample_rate = sf.read(wav_file)

    # pad input values and return pt tensor
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

    # INFERENCE
    # retrieve logits & take argmax
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # transcribe
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)

    return transcription

In [ ]:
# All Models :-

# Harveenchadha/vakyansh-wav2vec2-sanskrit-sam-60
# addy88/wav2vec2-base-sanskrit-84h (ERROR IN LOADING)
# addy88/wav2vec2-sanskrit-stt
# KabiAI/wav2vec2-large-xls-r-300m-Sanskrit-Hindi (ERROR IN LOADING)
# Harveenchadha/wav2vec2-pretrained-clsril-23-10k
# frmccann/CLSRIL-23



audio_input = testing_dataset[0]["speech"]
sample_rate = 16000
model = "addy88/wav2vec2-sanskrit-stt"

print("Predicted ==>\n", parse_transcription(model, wav_file=None, audio_input=audio_input, sample_rate=sample_rate))
print("\nActual ==>\n", testing_dataset[0]["target_text"])

Downloading:   0%|          | 0.00/213 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/730 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Predicted ==>
 नवशत्कोटि रुप्यकैः नगरस्य स्थापना भविष्यति

Actual ==>
 नवम् कोटी रक्तः नगरस्य स्थापनम् भविष्यति


# Uploading Files to drive or github

In [ ]:
!git clone https://huggingface.co/addy88/wav2vec2-sanskrit-stt

Cloning into 'wav2vec2-sanskrit-stt'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 46 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (46/46), done.
